In [159]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import date,datetime, timedelta
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier

In [160]:
speed = pd.read_csv('train.csv',index_col='date',parse_dates=['date'],dayfirst = True)
speed2 = pd.read_csv('test.csv',index_col='date',parse_dates=['date'],dayfirst = True)

In [161]:
speed = speed[['speed']]

In [162]:
speed_train1, speed_test1 = train_test_split(speed,test_size = 0.2,random_state = 3)
speed_train2, speed_test2 = train_test_split(speed,test_size = 0.2,random_state = 4)
speed_train3, speed_test3 = train_test_split(speed,test_size = 0.2,random_state = 5)
#train_X,test_X,train_y,test_y = train_test_split(train_data,train_target,test_size=0.3,random_state=5)

In [163]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    #df['quarter1'] = (df['date'].dt.quarter == 1).astype(int)
    #df['quarter2'] = (df['date'].dt.quarter == 2).astype(int)
    #df['quarter3'] = (df['date'].dt.quarter == 3).astype(int)
    #df['quarter4'] = (df['date'].dt.quarter == 4).astype(int)ata[(data.实际辐照度<mi)|(data.实际辐照度>ma)]
    df['night'] = ((df['date'].dt.hour<=6)|(df['date'].dt.hour>18)).astype(int)
    df['day'] = (df['night'] == 0).astype(int)
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year 
    #df['year1'] = (df['date'].dt.year == 2017).astype(int)
    #df['year2'] = (df['date'].dt.year == 2018).astype(int)
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','night','day','quarter','year','dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

In [164]:
X_train1, y_train1 = create_features(speed_train1, label='speed')
X_test1, y_test1 = create_features(speed_test1, label='speed')
X_train2, y_train2 = create_features(speed_train2, label='speed')
X_test2, y_test2 = create_features(speed_test2, label='speed')
X_train3, y_train3 = create_features(speed_train3, label='speed')
X_test3, y_test3 = create_features(speed_test3, label='speed')

<ipython-input-163-81e48100ae1f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df.index
<ipython-input-163-81e48100ae1f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = df['date'].dt.hour
<ipython-input-163-81e48100ae1f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [165]:
reg1 = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.1,gamma =0.3,max_depth=7,subsample = 1)

reg1.fit(X_train1, y_train1,
        early_stopping_rounds = 300,
        eval_set=[ (X_test1, y_test1)],
        verbose=True) # Change verbose to True if you want to see it train

[0]	validation_0-rmse:31.64051
Will train until validation_0-rmse hasn't improved in 300 rounds.
[1]	validation_0-rmse:28.54258
[2]	validation_0-rmse:25.75952
[3]	validation_0-rmse:23.26347
[4]	validation_0-rmse:21.02389
[5]	validation_0-rmse:19.01523
[6]	validation_0-rmse:17.21298
[7]	validation_0-rmse:15.60468
[8]	validation_0-rmse:14.16466
[9]	validation_0-rmse:12.88143
[10]	validation_0-rmse:11.73242
[11]	validation_0-rmse:10.71217
[12]	validation_0-rmse:9.80382
[13]	validation_0-rmse:9.00012
[14]	validation_0-rmse:8.29103
[15]	validation_0-rmse:7.66361
[16]	validation_0-rmse:7.11494
[17]	validation_0-rmse:6.63594
[18]	validation_0-rmse:6.22061
[19]	validation_0-rmse:5.85643
[20]	validation_0-rmse:5.53611
[21]	validation_0-rmse:5.26693
[22]	validation_0-rmse:5.03842
[23]	validation_0-rmse:4.83376
[24]	validation_0-rmse:4.66666
[25]	validation_0-rmse:4.51546
[26]	validation_0-rmse:4.39500
[27]	validation_0-rmse:4.29187
[28]	validation_0-rmse:4.20518
[29]	validation_0-rmse:4.12541
[3

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.3, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [166]:
### 性能度量
y_pred1 = reg1.predict(X_test1)
from sklearn.metrics import accuracy_score,mean_squared_error  # 准确率
accuracy = mean_squared_error(y_test1,y_pred1)
print("mse: %.2f%%"  %(accuracy))

mse: 10.55%


In [167]:
reg2 = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.07,max_depth=7,gamma =0.1,subsample = 0.8)

reg2.fit(X_train2, y_train2,
        early_stopping_rounds = 300,
        eval_set=[ (X_test2, y_test2)],
        verbose=True) # Change verbose to True if you want to see it train

[0]	validation_0-rmse:32.34502
Will train until validation_0-rmse hasn't improved in 300 rounds.
[1]	validation_0-rmse:30.11811
[2]	validation_0-rmse:28.05242
[3]	validation_0-rmse:26.13471
[4]	validation_0-rmse:24.35564
[5]	validation_0-rmse:22.70463
[6]	validation_0-rmse:21.17145
[7]	validation_0-rmse:19.75373
[8]	validation_0-rmse:18.43643
[9]	validation_0-rmse:17.21072
[10]	validation_0-rmse:16.07549
[11]	validation_0-rmse:15.02658
[12]	validation_0-rmse:14.05510
[13]	validation_0-rmse:13.15715
[14]	validation_0-rmse:12.32793
[15]	validation_0-rmse:11.56214
[16]	validation_0-rmse:10.85573
[17]	validation_0-rmse:10.20312
[18]	validation_0-rmse:9.60663
[19]	validation_0-rmse:9.05371
[20]	validation_0-rmse:8.54808
[21]	validation_0-rmse:8.08338
[22]	validation_0-rmse:7.65732
[23]	validation_0-rmse:7.26915
[24]	validation_0-rmse:6.91358
[25]	validation_0-rmse:6.59563
[26]	validation_0-rmse:6.30367
[27]	validation_0-rmse:6.03630
[28]	validation_0-rmse:5.79777
[29]	validation_0-rmse:5.57

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.07, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [168]:
### 性能度量
y_pred2 = reg2.predict(X_test2)
from sklearn.metrics import accuracy_score,mean_squared_error  # 准确率
accuracy = mean_squared_error(y_test2,y_pred2)
print("mse: %.2f%%"  %(accuracy))

mse: 11.30%


In [169]:
reg3 = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.1,max_depth=7,gamma =0.2,subsample = 0.7)

reg3.fit(X_train3, y_train3,
        early_stopping_rounds = 300,
        eval_set=[ (X_test3, y_test3)],
        verbose=True) # Change verbose to True if you want to see it train

[0]	validation_0-rmse:31.66377
Will train until validation_0-rmse hasn't improved in 300 rounds.
[1]	validation_0-rmse:28.57482
[2]	validation_0-rmse:25.79605
[3]	validation_0-rmse:23.30176
[4]	validation_0-rmse:21.06532
[5]	validation_0-rmse:19.05725
[6]	validation_0-rmse:17.25971
[7]	validation_0-rmse:15.65345
[8]	validation_0-rmse:14.20598
[9]	validation_0-rmse:12.91662
[10]	validation_0-rmse:11.76783
[11]	validation_0-rmse:10.75022
[12]	validation_0-rmse:9.83855
[13]	validation_0-rmse:9.03231
[14]	validation_0-rmse:8.32069
[15]	validation_0-rmse:7.69258
[16]	validation_0-rmse:7.13889
[17]	validation_0-rmse:6.65990
[18]	validation_0-rmse:6.23402
[19]	validation_0-rmse:5.86716
[20]	validation_0-rmse:5.54954
[21]	validation_0-rmse:5.26842
[22]	validation_0-rmse:5.02818
[23]	validation_0-rmse:4.82415
[24]	validation_0-rmse:4.65338
[25]	validation_0-rmse:4.50097
[26]	validation_0-rmse:4.37391
[27]	validation_0-rmse:4.26400
[28]	validation_0-rmse:4.17344
[29]	validation_0-rmse:4.09624
[3

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.2, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [170]:
### 性能度量
y_pred3 = reg3.predict(X_test3)
from sklearn.metrics import accuracy_score,mean_squared_error  # 准确率
accuracy = mean_squared_error(y_test3,y_pred3)
print("mse: %.2f%%"  %(accuracy))

mse: 11.30%


In [171]:
X_test, y_test = create_features(speed2, label='id')
X_test

,hour,dayofweek,night,day,quarter,year,dayofyear,dayofmonth,weekofyear
date,,,,,,,,,
2018-01-01 02:00:00,2,0,1,0,1,2018,1,1,1
2018-01-01 05:00:00,5,0,1,0,1,2018,1,1,1
2018-01-01 07:00:00,7,0,0,1,1,2018,1,1,1
2018-01-01 08:00:00,8,0,0,1,1,2018,1,1,1
2018-01-01 10:00:00,10,0,0,1,1,2018,1,1,1
...,...,...,...,...,...,...,...,...,...
2018-12-31 17:00:00,17,0,0,1,4,2018,365,31,1
2018-12-31 19:00:00,19,0,1,0,4,2018,365,31,1
2018-12-31 21:00:00,21,0,1,0,4,2018,365,31,1


In [173]:
#df.sort_index(ascending=False)
X_test.head(10)

,hour,dayofweek,night,day,quarter,year,dayofyear,dayofmonth,weekofyear
date,,,,,,,,,
2018-01-01 02:00:00,2,0,1,0,1,2018,1,1,1
2018-01-01 05:00:00,5,0,1,0,1,2018,1,1,1
2018-01-01 07:00:00,7,0,0,1,1,2018,1,1,1
2018-01-01 08:00:00,8,0,0,1,1,2018,1,1,1
2018-01-01 10:00:00,10,0,0,1,1,2018,1,1,1
2018-01-01 16:00:00,16,0,0,1,1,2018,1,1,1
2018-01-01 18:00:00,18,0,0,1,1,2018,1,1,1
2018-01-02 01:00:00,1,1,1,0,1,2018,2,2,1
2018-01-02 02:00:00,2,1,1,0,1,2018,2,2,1


In [174]:
speed_testing1 = reg1.predict(X_test)
speed_testing2 = reg2.predict(X_test)
speed_testing3 = reg3.predict(X_test)
#pjme_test['MW_Prediction'] = reg.predict(X_test)

In [175]:
speed_testing = 0.6 * speed_testing1 + 0.2 * speed_testing2 + 0.2 * speed_testing3

In [176]:
speed_testing = pd.DataFrame(speed_testing)
speed_testing.head(5)

,0
0,48.634399
1,47.961845
2,40.251205
3,31.168299
4,41.372059


In [177]:
Submission= pd.read_csv('sampleSubmission.csv')
Submission

,id,speed
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
3499,3499,0
3500,3500,0
3501,3501,0
3502,3502,0


In [178]:
Submission['speed'] = speed_testing
Submission

,id,speed
0,0,48.634399
1,1,47.961845
2,2,40.251205
3,3,31.168299
4,4,41.372059
...,...,...
3499,3499,9.706984
3500,3500,19.766598
3501,3501,46.988571
3502,3502,42.239861


In [179]:
Submission['speed'] = speed_testing
Submission

,id,speed
0,0,48.634399
1,1,47.961845
2,2,40.251205
3,3,31.168299
4,4,41.372059
...,...,...
3499,3499,9.706984
3500,3500,19.766598
3501,3501,46.988571
3502,3502,42.239861


In [180]:
Submission['speed'] = speed_testing
Submission

,id,speed
0,0,48.634399
1,1,47.961845
2,2,40.251205
3,3,31.168299
4,4,41.372059
...,...,...
3499,3499,9.706984
3500,3500,19.766598
3501,3501,46.988571
3502,3502,42.239861


In [157]:
Submission.to_csv('kaggle_xgboost_submission.csv',index=False,header = True)